In [1]:
import numpy as np
from numpy.linalg import inv, norm

def triangulate(Kl, Kr, Twl, Twr, pl, pr, Sl, Sr):
    """
    Triangulate 3D point position from camera projections.

    The function computes the 3D position of a point landmark from the 
    projection of the point into two camera images separated by a known
    baseline. All arrays should contain float64 values.

    Parameters:
    -----------
    Kl   - 3x3 np.array, left camera intrinsic calibration matrix.
    Kr   - 3x3 np.array, right camera intrinsic calibration matrix.
    Twl  - 4x4 np.array, homogeneous pose, left camera in world frame.
    Twr  - 4x4 np.array, homogeneous pose, right camera in world frame.
    pl   - 2x1 np.array, point in left camera image.
    pr   - 2x1 np.array, point in right camera image.
    Sl   - 2x2 np.array, left image point covariance matrix.
    Sr   - 2x2 np.array, right image point covariance matrix.

    Returns:
    --------
    Pl  - 3x1 np.array, closest point on ray from left camera  (in world frame).
    Pr  - 3x1 np.array, closest point on ray from right camera (in world frame).
    P   - 3x1 np.array, estimated 3D landmark position in the world frame.
    S   - 3x3 np.array, covariance matrix for estimated 3D point.
    """
    #--- FILL ME IN ---

    # Compute baseline (right camera translation minus left camera translation).

    # Unit vectors projecting from optical center to image plane points.
    # Use variables rayl and rayr for the rays.
 
    # Projected segment lengths.
    # Use variables ml and mr for the segment lengths.
    
    # Segment endpoints.
    # User variables Pl and Pr for the segment endpoints.

    # Now fill in with appropriate ray Jacobians. These are 
    # 3x4 matrices, but two columns are zeros (because the right
    # ray direction is not affected by the left image point and 
    # vice versa).
    drayl = np.zeros((3, 4))  # Jacobian left ray w.r.t. image points.
    drayr = np.zeros((3, 4))  # Jacobian right ray w.r.t. image points.

    # Add code here...
    #************************************************** 
    
    #compute unit vectors from optical center to image plane points, rayl and rayr
    temp = inv(Kl) @ np.vstack([pl,1]) 
    rl = Twl[0:3,0:3] @ temp
    rayl = rl / norm(rl)
    
    temp = inv(Kr) @ np.vstack([pr,1]) 
    rr = Twl[0:3,0:3] @ temp
    rayr = rr/norm(rr)
    
    #compute baseline, subtract translations
    baseline = Twr[0:3,3] - Twl[0:3,3] 
    
    #compute projected segment lengths
    temp = baseline @ rayl - (baseline @ rayr * rayl.T @ rayr)
    ml = temp / (1 -  (rayl.T @ rayr)**2 )
    mr = rayl.T @ rayr * ml - baseline @ rayr
    
    
    #segment endpoints Pl, Pr
    Pl = Twl[0:3,3:4] +rayl*ml
    Pr = Twr[0:3,3:4] + rayr*mr
    
    ##fill in with ray Jacobians
    
    
    ##compose dRl/du and dRl/du, partials of Rl with respect to u and v
    temp1 = Twl[0:3,0:3] @ inv(Kl)[0:3,0:1]    #these are the numerators
    temp2 = Twl[0:3,0:3] @ inv(Kl)[0:3,1:2]
    #denominators
    temp3 = (1/2)*((norm(rl) ** 2) ** (-1/2)) *(2*rl[0,0]*temp1[0,0] +  2*rl[1,0]*temp1[1,0] + 2*rl[2,0]*temp1[2,0])
    temp4 = (1/2)*((norm(rl) ** 2) ** (-1/2)) *(2*rl[0,0]*temp2[0,0] + 2*rl[1,0]*temp2[1,0] + 2*rl[2,0]*temp2[2,0])
    
    dRl_dul = (temp1 * norm(rl) - rl * temp3) / (norm(rl) ** 2)
    dRl_dvl = (temp2 * norm(rl) - rl * temp4) / (norm(rl) ** 2)
    
    #the partials of Rl with respect ur and vr are the same, simply 0 vectors
    dRl_dur = np.zeros((3,1))
    dRl_dvr = dRl_dur
    
    #now, with the above partials, we can put together drayl 
    drayl = np.hstack([dRl_dul,dRl_dvl,dRl_dur,dRl_dvr])
    
    ##repeat the above process for Rr
    
    t1 = Twr[0:3,0:3] @ inv(Kr)[0:3,0:1] 
    t2 = Twr[0:3,0:3] @ inv(Kr)[0:3,1:2]
    t3 = (1/2)*((norm(rr) ** 2 ) ** (-1/2)) *(2 * rr[0,0] * t1[0,0] +  2*rr[1,0]*t1[1,0] + 2*rr[2,0]*t1[2,0])
    t4 = (1/2)*((norm(rr) ** 2 ) ** (-1/2)) *  (2 * rr[0,0] * t2[0,0] +  2 * rr[1,0]*t2[1,0] + 2*rr[2,0]*t2[2,0])
    
    #dul and dvl
    dRr_dur = (t1 * norm(rr) - rr * t3) / (norm(rr) ** 2)
    dRr_dvr = (t2 * norm(rr) - rr * t4) / (norm(rr) ** 2)
        
    #the partials of Rr with respect ul and vl are the same, simply 0 vectors
    dRr_dul = np.zeros(shape = (3,1))
    dRr_dvl = dRr_dul
    
    #compose drayr with the above partials
    drayr = np.hstack([dRr_dul,dRr_dvl,dRr_dur,dRr_dvr])
  
    
    b = baseline
    #**************************************************
    #------------------

    # Compute dml and dmr (partials wrt segment lengths).
    u = np.dot(b.T, rayl) - np.dot(b.T, rayr)*np.dot(rayl.T, rayr)
    v = 1 - np.dot(rayl.T, rayr)**2

    du = (b.T@drayl).reshape(1, 4) - \
         (b.T@drayr).reshape(1, 4)*np.dot(rayl.T, rayr) - \
         np.dot(b.T, rayr)*((rayr.T@drayl) + (rayl.T@drayr)).reshape(1, 4)
 
    dv = -2*np.dot(rayl.T, rayr)*((rayr.T@drayl).reshape(1, 4) + \
        (rayl.T@drayr).reshape(1, 4))

    m = np.dot(b.T, rayr) - np.dot(b.T, rayl)@np.dot(rayl.T, rayr)
    n = np.dot(rayl.T, rayr)**2 - 1

    dm = (b.T@drayr).reshape(1, 4) - \
         (b.T@drayl).reshape(1, 4)*np.dot(rayl.T, rayr) - \
         np.dot(b.T, rayl)@((rayr.T@drayl) + (rayl.T@drayr)).reshape(1, 4)
    dn = -dv

    dml = (du*v - u*dv)/v**2
    dmr = (dm*n - m*dn)/n**2

    # Finally, compute Jacobian for P w.r.t. image points.
    JP = (ml*drayl + rayl*dml + mr*drayr + rayr*dmr)/2

    #--- FILL ME IN ---

    # 3D point.

    # 3x3 landmark point covariance matrix (need to form
    # the 4x4 image plane covariance matrix first).
    #**************************************************
    P = (Pl + Pr)/2
    
    cov = np.hstack([Sl, np.zeros((2,2))])
    cov = np.vstack( [cov, np.hstack([np.zeros((2,2)), Sr ])] )
    
    S= JP @ cov @ JP.T
    
    #**************************************************
    #------------------

    # Check for correct outputs...
    correct = isinstance(Pl, np.ndarray) and Pl.shape == (3, 1) and \
              isinstance(Pr, np.ndarray) and Pr.shape == (3, 1) and \
              isinstance(P,  np.ndarray) and P.shape  == (3, 1) and \
              isinstance(S,  np.ndarray) and S.shape  == (3, 3)

    if not correct:
        raise TypeError("Wrong type or size returned!")

    return Pl, Pr, P, S

In [13]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
#from dcm_from_rpy import dcm_from_rpy
#from triangulate import triangulate

# Camera intrinsic matrices.
Kl = np.array([[500.0, 0.0, 320], [0.0, 500.0, 240.0], [0, 0, 1]])
Kr = Kl

# Camera poses (left, right).
Twl = np.eye(4)
Twl[:3, :3] = dcm_from_rpy([-np.pi/2, 0, 0])  # Tilt for visualization.
Twr = Twl.copy()
Twr[0, 3] = 1.0  # Baseline.

# Image plane points (left, right).
pl = np.array([[360], [237.0]])
pr = np.array([[240], [238.5]])

# Image plane uncertainties (covariances).
Sl = np.eye(2)
Sr = np.eye(2)

[Pl, Pr, P, S] = triangulate(Kl, Kr, Twl, Twr, pl, pr, Sl, Sr)

# Visualize - plot rays and the estimate of P...
fig = plt.figure()
ax = fig.add_subplot(111, projection = '3d')

ax.plot(np.array([Twl[0, 3], Pl[0, 0]]), 
        np.array([Twl[1, 3], Pl[1, 0]]),
        np.array([Twl[2, 3], Pl[2, 0]]), 'b-')
ax.plot(np.array([Twr[0, 3], Pr[0, 0]]),
        np.array([Twr[1, 3], Pr[1, 0]]),
        np.array([Twr[2, 3], Pr[2, 0]]), 'r-')
ax.plot(np.array([Pl[0, 0], Pr[0, 0]]),
        np.array([Pl[1, 0], Pr[1, 0]]),
        np.array([Pl[2, 0], Pr[2, 0]]), 'g-')
ax.plot([P[0, 0]], [P[1, 0]], [P[2, 0]], 'bx', markersize = 8)
plt.show()

ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 3 and the array at index 1 has size 1

In [6]:
import numpy as np

def rpy_from_dcm(R):
    """
    Roll, pitch, yaw Euler angles from rotation matrix.

    The function computes roll, pitch and yaw angles from the
    rotation matrix R. The pitch angle p is constrained to the range
    (-pi/2, pi/2].  The returned angles are in radians.

    Inputs:
    -------
    R  - 3x3 orthonormal rotation matrix.

    Returns:
    --------
    rpy  - 3x1 np.array of roll, pitch, yaw Euler angles.
    """
    rpy = np.zeros((3, 1))

    # Roll.
    rpy[0] = np.arctan2(R[2, 1], R[2, 2])

    # Pitch.
    sp = -R[2, 0]
    cp = np.sqrt(R[0, 0]*R[0, 0] + R[1, 0]*R[1, 0])

    if np.abs(cp) > 1e-15:
      rpy[1] = np.arctan2(sp, cp)
    else:
      # Gimbal lock...
      rpy[1] = np.pi/2
  
      if sp < 0:
        rpy[1] = -rpy[1]

    # Yaw.
    rpy[2] = np.arctan2(R[1, 0], R[0, 0])

    return rpy

In [7]:
import numpy as np

def dcm_from_rpy(rpy):
    """
    Rotation matrix from roll, pitch, yaw Euler angles.

    The function produces a 3x3 orthonormal rotation matrix R
    from the vector rpy containing roll angle r, pitch angle p, and yaw angle
    y.  All angles are specified in radians.  We use the aerospace convention
    here (see descriptions below).  Note that roll, pitch and yaw angles are
    also often denoted by phi, theta, and psi (respectively).

    The angles are applied in the following order:

     1.  Yaw   -> by angle 'y' in the local (body-attached) frame.
     2.  Pitch -> by angle 'p' in the local frame.
     3.  Roll  -> by angle 'r' in the local frame.  

    Note that this is exactly equivalent to the following fixed-axis
    sequence:

     1.  Roll  -> by angle 'r' in the fixed frame.
     2.  Pitch -> by angle 'p' in the fixed frame.
     3.  Yaw   -> by angle 'y' in the fixed frame.

    Parameters:
    -----------
    rpy  - 3x1 np.array of roll, pitch, yaw Euler angles.

    Returns:
    --------
    R  - 3x3 np.array, orthonormal rotation matrix.
    """
    cr = np.cos(rpy[0]).item()
    sr = np.sin(rpy[0]).item()
    cp = np.cos(rpy[1]).item()
    sp = np.sin(rpy[1]).item()
    cy = np.cos(rpy[2]).item()
    sy = np.sin(rpy[2]).item()

    return np.array([[cy*cp, cy*sp*sr - sy*cr, cy*sp*cr + sy*sr],
                     [sy*cp, sy*sp*sr + cy*cr, sy*sp*cr - cy*sr],
                     [  -sp,            cp*sr,            cp*cr]])

In [11]:
#####SOLUTION
import numpy as np
from numpy.linalg import inv, norm

def triangulate(Kl, Kr, Twl, Twr, pl, pr, Sl, Sr):
    """
    Triangulate 3D point position from camera projections.
    The function computes the 3D position of a point landmark from the 
    projection of the point into two camera images separated by a known
    baseline.
    Parameters:
    -----------
    Kl   - 3 x 3 np.array, left camera intrinsic calibration matrix.
    Kr   - 3 x 3 np.array, right camera intrinsic calibration matrix.
    Twl  - 4x4 np.array, homogeneous pose, left camera in world frame.
    Twr  - 4x4 np.array, homogeneous pose, right camera in world frame.
    pl   - 2x1 np.array, point in left camera image.
    pr   - 2x1 np.array, point in right camera image.
    Sl   - 2x2 np.array, left image point covariance matrix.
    Sr   - 2x2 np.array, right image point covariance matrix.
    Returns:
    --------
    Pl  - 3x1 np.array, closest point on ray from left camera  (in world frame).
    Pr  - 3x1 np.array, closest point on ray from right camera (in world frame).
    P   - 3x1 np.array, estimated 3D landmark position in the world frame.
    S   - 3x3 np.array, covariance matrix for estimated 3D point.
    """
    # Initialize arrays for position and rotation
    rot_l = Twl[:3, :3]
    rot_r = Twr[:3, :3]
    pos_l = Twl[:3, 3]
    pos_r = Twr[:3, 3]

    # Compute baseline (right camera translation minus left camera translation).
    b = pos_r - pos_l

    # Unit vectors projecting from optical center to image plane points.
    # Use variables rayl and rayr for the rays.
    rayl_unnorm = rot_l @ inv(Kl) @ np.vstack((pl, [1]))
    rayr_unnorm = rot_r @ inv(Kr) @ np.vstack((pr, [1]))

    rayl = rayl_unnorm/norm(rayl_unnorm)
    rayr = rayr_unnorm/norm(rayr_unnorm)

    # Projected segment lengths.
    # Use variables ml and mr for the segment lengths.
    ml = (b.dot(rayl) - (b.dot(rayr))*((rayl.T).dot(rayr))) / (1-((rayl.T).dot(rayr)**2))
    mr = ((rayl.T).dot(rayr))*ml - b.dot(rayr)
    
    # Segment endpoints.
    # User variables Pl and Pr for the segment endpoints.
    Pl = pos_l.reshape(3,1) + rayl*ml
    Pr = pos_r.reshape(3,1) + rayr*mr

    # Now fill in with appropriate ray Jacobians. These are 
    # 3x4 matrices, but two columns are zeros (because the right
    # ray direction is not affected by the left image point and 
    # vice versa).
    drayl = np.zeros((3, 4))  # Jacobian left ray w.r.t. image points.
    drayr = np.zeros((3, 4))  # Jacobian right ray w.r.t. image points.

    # Calculate derivative of unit vector rays.
    # First get derivative of left and right rays (rl and rr) with respect
    # to u and v
    drl_du = (rot_l @ inv(Kl) @ np.array([[1, 0, 0]]).T).reshape((3,1))
    drl_dv = (rot_l @ inv(Kl) @ np.array([[0, 1, 0]]).T).reshape((3,1))
    drr_du = (rot_r @ inv(Kr) @ np.array([[1, 0, 0]]).T).reshape((3,1))
    drr_dv = (rot_r @ inv(Kr) @ np.array([[0, 1, 0]]).T).reshape((3,1))

    # Get magnitude of ray for denominator value in derivative
    magl = norm(rayl_unnorm)
    magr = norm(rayr_unnorm)

    # Calculate derivative of norm(rayl) and norm(rayr) (nrl and nrr)
    # with respect to u and v
    dnrl_du = drl_du.T @ rayl_unnorm / magl
    dnrl_dv = drl_dv.T @ rayl_unnorm / magl
    dnrr_du = drr_du.T @ rayr_unnorm / magr
    dnrr_dv = drr_dv.T @ rayr_unnorm / magr

    # Calculate the final derivative using chain rule
    drayl[:, 0] = ((drl_du*magl - rayl*dnrl_du) / magl**2).reshape((3,))
    drayl[:, 1] = ((drl_dv*magl - rayl*dnrl_dv) / magl**2).reshape((3,))
    drayr[:, 2] = ((drr_du*magr - rayr*dnrr_du) / magr**2).reshape((3,))
    drayr[:, 3] = ((drr_dv*magr - rayr*dnrr_dv) / magr**2).reshape((3,))

    #------------------

    # Compute dml and dmr (partials wrt segment lengths).
    u = np.dot(b.T, rayl) - np.dot(b.T, rayr)*np.dot(rayl.T, rayr)
    v = 1 - np.dot(rayl.T, rayr)**2

    du = (b.T@drayl).reshape(1, 4) - \
         (b.T@drayr).reshape(1, 4)*np.dot(rayl.T, rayr) - \
         np.dot(b.T, rayr)*((rayr.T@drayl) + (rayl.T@drayr)).reshape(1, 4)
 
    dv = -2*np.dot(rayl.T, rayr)*((rayr.T@drayl).reshape(1, 4) + \
        (rayl.T@drayr).reshape(1, 4))

    m = np.dot(b.T, rayr) - np.dot(b.T, rayl)@np.dot(rayl.T, rayr)
    n = np.dot(rayl.T, rayr)**2 - 1

    dm = (b.T@drayr).reshape(1, 4) - \
         (b.T@drayl).reshape(1, 4)*np.dot(rayl.T, rayr) - \
         np.dot(b.T, rayl)@((rayr.T@drayl) + (rayl.T@drayr)).reshape(1, 4)
    dn = -dv

    dml = (du*v - u*dv)/v**2
    dmr = (dm*n - m*dn)/n**2

    # Finally, compute Jacobian for P w.r.t. image points.
    JP = (ml*drayl + rayl*dml + mr*drayr + rayr*dmr)/2

    # 3D point.
    P = (Pl+Pr)/2
    # 3x3 landmark point covariance matrix (need to form
    # the 4x4 image plane covariance matrix first).
    S_im = np.zeros((4,4))
    S_im[0:2, 0:2] = Sl
    S_im[2:4, 2:4] = Sr
    S = JP @ S_im @ JP.T

    return Pl, Pr, P, S

In [12]:
import numpy as np
from numpy.linalg import inv, norm

def triangulate(Kl, Kr, Twl, Twr, pl, pr, Sl, Sr):
    """
    Triangulate 3D point position from camera projections.

    The function computes the 3D position of a point landmark from the 
    projection of the point into two camera images separated by a known
    baseline. All arrays should contain float64 values.

    Parameters:
    -----------
    Kl   - 3x3 np.array, left camera intrinsic calibration matrix.
    Kr   - 3x3 np.array, right camera intrinsic calibration matrix.
    Twl  - 4x4 np.array, homogeneous pose, left camera in world frame.
    Twr  - 4x4 np.array, homogeneous pose, right camera in world frame.
    pl   - 2x1 np.array, point in left camera image.
    pr   - 2x1 np.array, point in right camera image.
    Sl   - 2x2 np.array, left image point covariance matrix.
    Sr   - 2x2 np.array, right image point covariance matrix.

    Returns:
    --------
    Pl  - 3x1 np.array, closest point on ray from left camera  (in world frame).
    Pr  - 3x1 np.array, closest point on ray from right camera (in world frame).
    P   - 3x1 np.array, estimated 3D landmark position in the world frame.
    S   - 3x3 np.array, covariance matrix for estimated 3D point.
    """
    #--- FILL ME IN ---

    # Compute baseline (right camera translation minus left camera translation).

    # Unit vectors projecting from optical center to image plane points.
    # Use variables rayl and rayr for the rays.
 
    # Projected segment lengths.
    # Use variables ml and mr for the segment lengths.
    
    # Segment endpoints.
    # User variables Pl and Pr for the segment endpoints.

    # Now fill in with appropriate ray Jacobians. These are 
    # 3x4 matrices, but two columns are zeros (because the right
    # ray direction is not affected by the left image point and 
    # vice versa).
    drayl = np.zeros((3, 4))  # Jacobian left ray w.r.t. image points.
    drayr = np.zeros((3, 4))  # Jacobian right ray w.r.t. image points.

    # Add code here...
    #************************************************** 
    #rotation matrices
    Rl = Twl[0:3,0:3]
    Rr = Twr[0:3,0:3]
    #position vectors
    pl = Twl[0:3,3]
    pr = Twr[0:3,3]
    #compute unit vectors from optical center to image plane points, rayl and rayr
    
    #temp = inv(Kl) @ np.vstack([pl,[1]]) 
    rl = Rl @ inv(Kl) @ np.vstack([pl,1])
    rayl = rl / norm(rl)
    
    #temp = inv(Kr) @ np.vstack([pr,1]) 
    rr = Rr @ inv(Kr) @ np.vstack([pr,1])
    rayr = rr/norm(rr)
    
    #compute baseline, subtract translations
    baseline = pr - pl
    
    #compute projected segment lengths
    temp = baseline @ rayl - (baseline @ rayr * rayl.T @ rayr)
    ml = temp / (1 -  (rayl.T @ rayr)**2 )
    mr = rayl.T @ rayr * ml - baseline @ rayr
    
    
    #segment endpoints Pl, Pr
    Pl = Twl[0:3,3:4] +rayl*ml
    Pr = Twr[0:3,3:4] + rayr*mr
    
    ##fill in with ray Jacobians
    
    
    ##compose dRl/du and dRl/du, partials of Rl with respect to u and v
    temp1 = Twl[0:3,0:3] @ inv(Kl)[0:3,0:1]    #these are the numerators
    temp2 = Twl[0:3,0:3] @ inv(Kl)[0:3,1:2]
    #denominators
    temp3 = (1/2)*((norm(rl) ** 2) ** (-1/2)) *(2*rl[0,0]*temp1[0,0] +  2*rl[1,0]*temp1[1,0] + 2*rl[2,0]*temp1[2,0])
    temp4 = (1/2)*((norm(rl) ** 2) ** (-1/2)) *(2*rl[0,0]*temp2[0,0] + 2*rl[1,0]*temp2[1,0] + 2*rl[2,0]*temp2[2,0])
    
    dRl_dul = (temp1 * norm(rl) - rl * temp3) / (norm(rl) ** 2)
    dRl_dvl = (temp2 * norm(rl) - rl * temp4) / (norm(rl) ** 2)
    
    #the partials of Rl with respect ur and vr are the same, simply 0 vectors
    dRl_dur = np.zeros((3,1))
    dRl_dvr = dRl_dur
    
    #now, with the above partials, we can put together drayl 
    drayl = np.hstack([dRl_dul,dRl_dvl,dRl_dur,dRl_dvr])
    
    ##repeat the above process for Rr
    
    t1 = Twr[0:3,0:3] @ inv(Kr)[0:3,0:1] 
    t2 = Twr[0:3,0:3] @ inv(Kr)[0:3,1:2]
    t3 = (1/2)*((norm(rr) ** 2 ) ** (-1/2)) *(2 * rr[0,0] * t1[0,0] +  2*rr[1,0]*t1[1,0] + 2*rr[2,0]*t1[2,0])
    t4 = (1/2)*((norm(rr) ** 2 ) ** (-1/2)) *  (2 * rr[0,0] * t2[0,0] +  2 * rr[1,0]*t2[1,0] + 2*rr[2,0]*t2[2,0])
    
    #dul and dvl
    dRr_dur = (t1 * norm(rr) - rr * t3) / (norm(rr) ** 2)
    dRr_dvr = (t2 * norm(rr) - rr * t4) / (norm(rr) ** 2)
        
    #the partials of Rr with respect ul and vl are the same, simply 0 vectors
    dRr_dul = np.zeros(shape = (3,1))
    dRr_dvl = dRr_dul
    
    #compose drayr with the above partials
    drayr = np.hstack([dRr_dul,dRr_dvl,dRr_dur,dRr_dvr])
  
    
    b = baseline
    #**************************************************
    #------------------

    # Compute dml and dmr (partials wrt segment lengths).
    u = np.dot(b.T, rayl) - np.dot(b.T, rayr)*np.dot(rayl.T, rayr)
    v = 1 - np.dot(rayl.T, rayr)**2

    du = (b.T@drayl).reshape(1, 4) - \
         (b.T@drayr).reshape(1, 4)*np.dot(rayl.T, rayr) - \
         np.dot(b.T, rayr)*((rayr.T@drayl) + (rayl.T@drayr)).reshape(1, 4)
 
    dv = -2*np.dot(rayl.T, rayr)*((rayr.T@drayl).reshape(1, 4) + \
        (rayl.T@drayr).reshape(1, 4))

    m = np.dot(b.T, rayr) - np.dot(b.T, rayl)@np.dot(rayl.T, rayr)
    n = np.dot(rayl.T, rayr)**2 - 1

    dm = (b.T@drayr).reshape(1, 4) - \
         (b.T@drayl).reshape(1, 4)*np.dot(rayl.T, rayr) - \
         np.dot(b.T, rayl)@((rayr.T@drayl) + (rayl.T@drayr)).reshape(1, 4)
    dn = -dv

    dml = (du*v - u*dv)/v**2
    dmr = (dm*n - m*dn)/n**2

    # Finally, compute Jacobian for P w.r.t. image points.
    JP = (ml*drayl + rayl*dml + mr*drayr + rayr*dmr)/2

    #--- FILL ME IN ---

    # 3D point.

    # 3x3 landmark point covariance matrix (need to form
    # the 4x4 image plane covariance matrix first).
    #**************************************************
    P = (Pl + Pr)/2
    
    cov = np.zeros((4,4))
    cov[0:2, 0:2] = Sl
    cov[2:4, 2:4] = Sr
    #cov = np.hstack([Sl, np.zeros((2,2))])
    #cov = np.vstack( [cov, np.hstack([np.zeros((2,2)), Sr ])] )
    
    S= JP @ cov @ JP.T
    
    #**************************************************
    #------------------

    # Check for correct outputs...
    correct = isinstance(Pl, np.ndarray) and Pl.shape == (3, 1) and \
              isinstance(Pr, np.ndarray) and Pr.shape == (3, 1) and \
              isinstance(P,  np.ndarray) and P.shape  == (3, 1) and \
              isinstance(S,  np.ndarray) and S.shape  == (3, 3)

    if not correct:
        raise TypeError("Wrong type or size returned!")

    return Pl, Pr, P, S